<a href="https://colab.research.google.com/github/bala-ceg/backtest-strategies/blob/main/weeklyexpiry_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import Necessary Libs

In [1]:
!pip install nsepython

In [2]:
from nsepython import *

## NIFTY OI sample snippet

In [3]:
symbol = "NIFTY"
start_date = "07-06-2024"
end_date ="12-06-2024"
instrumentType = "options"
expiry_date ="13-Jun-2024"
put_df = derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice="",optionType="PE")
call_df = derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice="",optionType="CE")
if not put_df.empty:
  put_df['FH_OPEN_INT'] = put_df['FH_OPEN_INT'].astype(int)
  put_df['FH_TOT_TRADED_QTY'] = put_df['FH_TOT_TRADED_QTY'].astype(int)
  put_df = put_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
  underlying_value = put_df['FH_UNDERLYING_VALUE'].iloc[0]

if not call_df.empty:
  call_df['FH_OPEN_INT'] = call_df['FH_OPEN_INT'].astype(int)
  call_df['FH_TOT_TRADED_QTY'] = call_df['FH_TOT_TRADED_QTY'].astype(int)
  call_df = call_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
  underlying_value = call_df['FH_UNDERLYING_VALUE'].iloc[0]

In [4]:
put_total_open_int = put_df['FH_OPEN_INT'].sum()
put_total_volume = put_df['FH_TOT_TRADED_QTY'].sum()

put_df['FH_OPEN_INT_PERCENTAGE'] = (put_df['FH_OPEN_INT'] / put_total_open_int) * 100

put_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','FH_OPEN_INT_PERCENTAGE']].head(5)

,FH_STRIKE_PRICE,FH_OPEN_INT,FH_OPEN_INT_PERCENTAGE
30,23000.00,8672575,11.104410
26,23200.00,6306300,8.074619
40,22000.00,5141825,6.583620
33,22700.00,4899450,6.273281
35,22600.00,4480375,5.736696


In [5]:
put_df['FH_STRIKE_PRICE'] = put_df['FH_STRIKE_PRICE'].astype(float)
put_df['ABS_DIFF'] = abs(put_df['FH_STRIKE_PRICE'] - float(underlying_value))

sorted_put_df = put_df.sort_values(by='ABS_DIFF')

closest_rows = sorted_put_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]

print(closest_rows.head(3))
print(underlying_value)
put_df['FH_OPEN_INT'].idxmax()
row_with_max_open_int = put_df.loc[put_df['FH_OPEN_INT'].idxmax()]
row_with_max_open_int['FH_OPEN_INT']
near_strike_put_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
print(near_strike_put_open_int)
near_strike_put_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()
print(near_strike_put_vol)

    FH_STRIKE_PRICE  FH_OPEN_INT  ABS_DIFF  FH_OPEN_INT_PERCENTAGE  \
24          23300.0      4479850     22.95                5.736023   
23          23350.0      2018600     27.05                2.584626   
25          23250.0      1960200     72.95                2.509850   

    FH_TOT_TRADED_QTY  
24          146726850  
23           97113150  
25           54860850  
23322.95
8458650
298700850


In [6]:
call_total_open_int = call_df['FH_OPEN_INT'].sum()
call_total_volume = call_df['FH_TOT_TRADED_QTY'].sum()

call_df['FH_OPEN_INT_PERCENTAGE'] = (call_df['FH_OPEN_INT'] / call_total_open_int) * 100

call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','FH_OPEN_INT_PERCENTAGE']].head(5)


,FH_STRIKE_PRICE,FH_OPEN_INT,FH_OPEN_INT_PERCENTAGE
15,23800.00,6778900,10.138155
6,24500.00,6096750,9.117968
18,23600.00,5065475,7.575649
16,23700.00,5017825,7.504387
22,23300.00,3894175,5.823917


In [7]:
call_df['FH_STRIKE_PRICE'] = call_df['FH_STRIKE_PRICE'].astype(float)
call_df['ABS_DIFF'] = abs(call_df['FH_STRIKE_PRICE'] - float(underlying_value))

# Sort by the absolute difference
sorted_call_df = call_df.sort_values(by='ABS_DIFF')

# Select the rows that are closest to the underlying value
closest_rows = sorted_call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]

print(closest_rows.head(3))
print(underlying_value)
call_df['FH_OPEN_INT'].idxmax()
row_with_max_open_int = call_df.loc[call_df['FH_OPEN_INT'].idxmax()]
row_with_max_open_int['FH_OPEN_INT']
near_strike_call_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
print(near_strike_call_open_int)
near_strike_call_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()
print(near_strike_call_vol)

    FH_STRIKE_PRICE  FH_OPEN_INT  ABS_DIFF  FH_OPEN_INT_PERCENTAGE  \
22          23300.0      3894175     22.95                5.823917   
21          23350.0      2703625     27.05                4.043395   
23          23250.0       803825     72.95                1.202157   

    FH_TOT_TRADED_QTY  
22          104475950  
21          105632000  
23           21820550  
23322.95
7401625
231928500


In [8]:
pcr = float(put_total_open_int/call_total_open_int)
print("PCR OI:",pcr)
pcr_vol = float(put_total_volume/call_total_volume)
print("PCR volume:",pcr_vol)

PCR OI: 1.1680253076244043
PCR volume: 1.346010664870468


In [9]:
near_pcr = float(near_strike_put_open_int/near_strike_call_open_int)
print("Near PCR OI:",near_pcr)
near_pcr_vol = float(near_strike_put_vol/near_strike_call_vol)
print("Near PCR volume:",near_pcr_vol)

Near PCR OI: 1.142809855943796
Near PCR volume: 1.2879005814291904


In [10]:
call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE']].head(15)

,FH_STRIKE_PRICE,FH_OPEN_INT,ABS_DIFF,FH_OPEN_INT_PERCENTAGE
15,23800.0,6778900,477.05,10.138155
6,24500.0,6096750,1177.05,9.117968
18,23600.0,5065475,277.05,7.575649
16,23700.0,5017825,377.05,7.504387
22,23300.0,3894175,22.95,5.823917
14,23850.0,3164200,527.05,4.732206
20,23450.0,3091550,127.05,4.623554
21,23350.0,2703625,27.05,4.043395
17,23650.0,2701800,327.05,4.040665
13,23900.0,2663700,577.05,3.983685


In [11]:
call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT']]

,FH_STRIKE_PRICE,FH_OPEN_INT
15,23800.0,6778900
6,24500.0,6096750
18,23600.0,5065475
16,23700.0,5017825
22,23300.0,3894175
...,...,...
62,20600.0,1700
61,20650.0,1400
56,21250.0,875
59,20900.0,775


In [173]:
import pandas as pd
historic_nifty_df = pd.read_csv('/content/Nifty50_5Ydata.csv')
historic_nifty_df['Date'] = pd.to_datetime(historic_nifty_df['Date'], format="%d-%m-%Y")
numeric_columns = ["Open", "High", "Low", "Price"]
for column in numeric_columns:
    historic_nifty_df[column] = historic_nifty_df[column].str.replace(",", "").astype(float)
historic_nifty_df['Change %'] = historic_nifty_df['Change %'].str.replace("%", "").astype(float)
historic_nifty_df

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-06-20,23567.00,23586.15,23624.00,23442.60,280.34B,0.22
1,2024-06-19,23516.00,23629.85,23664.00,23412.90,328.81B,-0.18
2,2024-06-18,23557.90,23570.80,23579.05,23499.70,272.21B,0.39
3,2024-06-14,23465.60,23464.95,23490.40,23334.25,223.09B,0.29
4,2024-06-13,23398.90,23480.95,23481.05,23353.90,268.28B,0.33
...,...,...,...,...,...,...,...
1290,2019-04-04,11598.00,11660.20,11662.55,11559.20,349.03B,-0.39
1291,2019-04-03,11643.95,11735.30,11761.00,11629.15,365.76B,-0.59
1292,2019-04-02,11713.20,11711.55,11729.35,11655.85,386.13B,0.38
1293,2019-04-01,11669.15,11665.20,11738.10,11644.75,379.57B,0.39


In [174]:
data = historic_nifty_df.copy()
data['Day_of_Week'] = data['Date'].dt.dayofweek
data = data[['Date', 'Price', 'Day_of_Week','Change %']]
data['Week_Number'] = data['Date'].dt.isocalendar().week
data['Year'] = data['Date'].dt.isocalendar().year

day_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday'}
data['Day_of_Week'] = data['Day_of_Week'].map(day_map)

pivoted_data_1 = data.pivot_table(index=['Year', 'Week_Number'], columns='Day_of_Week', values='Price')
pivoted_data_2 = data.pivot_table(index=['Year', 'Week_Number'], columns='Day_of_Week', values='Change %')

In [153]:
import pandas as pd
from datetime import timedelta


start_date = pd.to_datetime("29-03-2019", format="%d-%m-%Y")
end_date = pd.to_datetime("20-06-2024", format="%d-%m-%Y")

start_list = []
end_list = []
target_list = []
years = []
week_numbers = []



current_date = start_date
while current_date <= end_date:
    if current_date.weekday() == 4:
        start_list.append(current_date.strftime("%d-%m-%Y"))
    if current_date.weekday() == 3:
        end_list.append(current_date.strftime("%d-%b-%Y"))
        years.append(current_date.isocalendar().year)
        week_numbers.append(current_date.isocalendar().week)
    if current_date.weekday() == 2:
        try :
          historic_nifty_df[historic_nifty_df['Date'] == current_date.strftime("%d-%m-%Y") ]['Price'].iloc[0]
          target_list.append(current_date.strftime("%d-%m-%Y"))
        except:
          previous_date = current_date - timedelta(days=1)
          target_list.append(previous_date.strftime("%d-%m-%Y"))

    current_date += timedelta(days=1)

print("Start List (Fridays):", start_list)
print("End List (Thursdays):", end_list)
print("Target List (Wednesdays):", target_list)
print(years)
print(week_numbers)

Start List (Fridays): ['29-03-2019', '05-04-2019', '12-04-2019', '19-04-2019', '26-04-2019', '03-05-2019', '10-05-2019', '17-05-2019', '24-05-2019', '31-05-2019', '07-06-2019', '14-06-2019', '21-06-2019', '28-06-2019', '05-07-2019', '12-07-2019', '19-07-2019', '26-07-2019', '02-08-2019', '09-08-2019', '16-08-2019', '23-08-2019', '30-08-2019', '06-09-2019', '13-09-2019', '20-09-2019', '27-09-2019', '04-10-2019', '11-10-2019', '18-10-2019', '25-10-2019', '01-11-2019', '08-11-2019', '15-11-2019', '22-11-2019', '29-11-2019', '06-12-2019', '13-12-2019', '20-12-2019', '27-12-2019', '03-01-2020', '10-01-2020', '17-01-2020', '24-01-2020', '31-01-2020', '07-02-2020', '14-02-2020', '21-02-2020', '28-02-2020', '06-03-2020', '13-03-2020', '20-03-2020', '27-03-2020', '03-04-2020', '10-04-2020', '17-04-2020', '24-04-2020', '01-05-2020', '08-05-2020', '15-05-2020', '22-05-2020', '29-05-2020', '05-06-2020', '12-06-2020', '19-06-2020', '26-06-2020', '03-07-2020', '10-07-2020', '17-07-2020', '24-07-2020

In [15]:
# pcr_data_df = pd.DataFrame(columns=['Year', 'Week_Number', 'pcr', 'pcr_vol', 'near_pcr',  'near_pcr_vol'])
# #pcr_data_df = pd.DataFrame()

# for start_date, target_date, end_date,year,week_num in zip(start_list, target_list, end_list,years,week_numbers):
#       symbol = "NIFTY"
#       instrumentType = "options"
#       print(start_date, target_date, end_date,year,week_num)
#       put_df = derivative_history(symbol,start_date,target_date,instrumentType,end_date,strikePrice="",optionType="PE")
#       call_df = derivative_history(symbol,start_date,target_date,instrumentType,end_date,strikePrice="",optionType="CE")
#       try:
#           underlying_value = put_df['FH_UNDERLYING_VALUE'].iloc[0]
#           print(underlying_value)

#           if not put_df.empty:
#             put_df['FH_OPEN_INT'] = put_df['FH_OPEN_INT'].astype(int)
#             put_df['FH_TOT_TRADED_QTY'] = put_df['FH_TOT_TRADED_QTY'].astype(int)
#             put_df = put_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
#             put_total_open_int = put_df['FH_OPEN_INT'].sum()
#             put_total_volume = put_df['FH_TOT_TRADED_QTY'].sum()
#             put_df['FH_OPEN_INT_PERCENTAGE'] = (put_df['FH_OPEN_INT'] / put_total_open_int) * 100
#             put_df['FH_STRIKE_PRICE'] = put_df['FH_STRIKE_PRICE'].astype(float)
#             put_df['ABS_DIFF'] = abs(put_df['FH_STRIKE_PRICE'] - float(underlying_value))

#             sorted_put_df = put_df.sort_values(by='ABS_DIFF')

#             closest_rows = sorted_put_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]
#             put_df['FH_OPEN_INT'].idxmax()
#             row_with_max_open_int = put_df.loc[put_df['FH_OPEN_INT'].idxmax()]
#             row_with_max_open_int['FH_OPEN_INT']
#             near_strike_put_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
#             near_strike_put_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()

#           if not call_df.empty:
#             call_df['FH_OPEN_INT'] = call_df['FH_OPEN_INT'].astype(int)
#             call_df['FH_TOT_TRADED_QTY'] = call_df['FH_TOT_TRADED_QTY'].astype(int)
#             call_df = call_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
#             call_total_open_int = call_df['FH_OPEN_INT'].sum()
#             call_total_volume = call_df['FH_TOT_TRADED_QTY'].sum()

#             call_df['FH_OPEN_INT_PERCENTAGE'] = (call_df['FH_OPEN_INT'] / call_total_open_int) * 100
#             call_df['FH_STRIKE_PRICE'] = call_df['FH_STRIKE_PRICE'].astype(float)
#             call_df['ABS_DIFF'] = abs(call_df['FH_STRIKE_PRICE'] - float(underlying_value))

#             sorted_call_df = call_df.sort_values(by='ABS_DIFF')
#             closest_rows = sorted_call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]

#             call_df['FH_OPEN_INT'].idxmax()
#             row_with_max_open_int = call_df.loc[call_df['FH_OPEN_INT'].idxmax()]
#             row_with_max_open_int['FH_OPEN_INT']
#             near_strike_call_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
#             near_strike_call_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()

#           if not put_df.empty and not call_df.empty:
#             pcr = float(put_total_open_int/call_total_open_int)
#             pcr_vol = float(put_total_volume/call_total_volume)
#             near_pcr = float(near_strike_put_open_int/near_strike_call_open_int)
#             near_pcr_vol = float(near_strike_put_vol/near_strike_call_vol)
#             new_data = {'Year': year,'Week_Number': week_num,'pcr': pcr,'pcr_vol': pcr_vol,'near_pcr': near_pcr,'near_pcr_vol': near_pcr_vol}
#             print(new_data)
#             pcr_data_df = pd.concat([pcr_data_df, pd.DataFrame([new_data])], ignore_index=True)
#           # else:
#           #   new_data = {'Year': year,'Week_Number': week_num,'pcr': None,'pcr_vol': None,'near_pcr': None,'near_pcr_vol': None}
#           #   pcr_data_df = pd.concat([pcr_data_df, pd.DataFrame([new_data])], ignore_index=True)
#       except:
#           print("error occured")



In [175]:
pcr_data_df = pd.read_csv('/content/pcr_data.csv')


In [176]:
pcr_data_df

,Year,Week_Number,pcr,pcr_vol,near_pcr,near_pcr_vol
0,2019,14,1.268668,0.870815,1.068206,0.794347
1,2019,15,0.692563,0.823201,0.890929,1.661440
2,2019,16,1.632566,1.054964,1.061811,0.688205
3,2019,17,1.457176,0.893068,0.580621,0.275643
4,2019,19,0.318322,0.671321,0.872058,2.741546
...,...,...,...,...,...,...
255,2024,21,1.100094,0.835847,0.913113,0.528212
256,2024,22,0.704400,1.171862,0.979367,1.426766
257,2024,23,0.535708,0.551546,0.791132,0.241626
258,2024,24,0.723595,0.697100,0.770853,0.528594


In [177]:
pivoted_data_1.columns.name = None
pivoted_data_1.reset_index(inplace=True)


pivoted_data_2.columns.name = None
pivoted_data_2.reset_index(inplace=True)

pivoted_data_1['Friday'] = pivoted_data_1['Friday'].shift(1)

pivoted_data_1 = pivoted_data_1[['Year','Week_Number', 'Friday', 'Monday', 'Tuesday',  'Wednesday','Thursday']]
pivoted_data_1 = pivoted_data_1.dropna(thresh=pivoted_data_1.shape[1] - 5 + 1)
pivoted_data_1['Thursday'].fillna(pivoted_data_1['Wednesday'], inplace=True)

pivoted_data_2['Friday'] = pivoted_data_2['Friday'].shift(1)

pivoted_data_2 = pivoted_data_2[['Year','Week_Number', 'Friday', 'Monday', 'Tuesday',  'Wednesday','Thursday']]
pivoted_data_2 = pivoted_data_2.dropna(thresh=pivoted_data_2.shape[1] - 5 + 1)

In [178]:
pivoted_data_2.to_csv('pivoted_data_2.csv', index=False)

In [158]:
print(pivoted_data_2)

     Year  Week_Number  Friday  Monday  Tuesday  Wednesday  Thursday
1    2019           14    0.47    0.39     0.38      -0.59     -0.39
2    2019           15    0.59   -0.53     0.58      -0.75      0.11
3    2019           16    0.40    0.40     0.83        NaN     -0.29
4    2019           17     NaN   -1.35    -0.16       1.30     -0.72
5    2019           18    0.97     NaN    -0.06        NaN     -0.20
..    ...          ...     ...     ...      ...        ...       ...
269  2024           21    0.28     NaN     0.12       0.31      1.64
270  2024           22   -0.05   -0.11    -0.19      -0.80     -0.95
271  2024           23    0.19    3.25    -5.93       3.36      0.89
272  2024           24    2.05   -0.13     0.02       0.25      0.33
273  2024           25    0.29     NaN     0.39      -0.18      0.22

[273 rows x 7 columns]


In [179]:
pivoted_data_2['Friday'].fillna(0, inplace=True)
pivoted_data_2['Monday'].fillna(0, inplace=True)
pivoted_data_2['Tuesday'].fillna(0, inplace=True)
pivoted_data_2['Wednesday'].fillna(0, inplace=True)

In [180]:
def identify_trend(row):
    trends = []
    if row['Friday'] > 0.1:
        trends.append('Up')
    elif row['Friday'] < -0.1:
        trends.append('Down')
    else:
        trends.append('Flat')

    if row['Monday'] > 0.1:
        trends.append('Up')
    elif row['Monday'] < -0.1:
        trends.append('Down')
    else:
        trends.append('Flat')

    if row['Tuesday'] > 0.1:
        trends.append('Up')
    elif row['Tuesday'] < -0.1:
        trends.append('Down')
    else:
        trends.append('Flat')

    if row['Wednesday'] > 0.1:
        trends.append('Up')
    elif row['Wednesday'] < -0.1:
        trends.append('Down')
    else:
        trends.append('Flat')


    return trends

def expected_trend(row):

    if row['Thursday'] > 0.1:
        return('Up')
    elif row['Thursday'] < -0.1:
        return('Down')
    else:
        return('Flat')

def consensus_trend(trend_list):
    trends = trend_list['Trend']
    count = sum(1 if trend == 'Up' else -1 if trend == 'Down' else 0 for trend in trends)
    if count > 0:
        return 'Up'
    elif count < 0:
        return 'Down'
    else:
        return 'Flat'

def lastday_trend(row):

    if row['Wednesday'] > 0.1:
        return('Up')
    elif row['Thursday'] < -0.1:
        return('Down')
    else:
        return('Flat')

pivoted_data_2['Trend'] = pivoted_data_2.apply(identify_trend, axis=1)
pivoted_data_2['Expected'] = pivoted_data_2.apply(expected_trend, axis=1)
pivoted_data_2['Consenus_Trend'] = pivoted_data_2.apply(consensus_trend, axis=1)
pivoted_data_2['lastday_Trend'] = pivoted_data_2.apply(lastday_trend, axis=1)

In [181]:
pivoted_data_2.head(5)

,Year,Week_Number,Friday,Monday,Tuesday,Wednesday,Thursday,Trend,Expected,Consenus_Trend,lastday_Trend
1,2019,14,0.47,0.39,0.38,-0.59,-0.39,"[Up, Up, Up, Down]",Down,Up,Down
2,2019,15,0.59,-0.53,0.58,-0.75,0.11,"[Up, Down, Up, Down]",Up,Flat,Flat
3,2019,16,0.40,0.40,0.83,0.00,-0.29,"[Up, Up, Up, Flat]",Down,Up,Down
4,2019,17,0.00,-1.35,-0.16,1.30,-0.72,"[Flat, Down, Down, Up]",Down,Down,Up
5,2019,18,0.97,0.00,-0.06,0.00,-0.20,"[Up, Flat, Flat, Flat]",Down,Up,Down


In [182]:
pivoted_data_2['Year_Week'] = pivoted_data_2['Year'].astype(str) + '_' + pivoted_data_2['Week_Number'].astype(str)
pivoted_data_3 = pivoted_data_2
pcr_data_df['Year_Week']  = pcr_data_df['Year'].astype(str) + '_' + pcr_data_df['Week_Number'].astype(str)
pcr_data_df_1 = pcr_data_df

result = pd.merge(pivoted_data_3, pcr_data_df_1, on='Year_Week', how='inner')
print(result.head(5))

   Year_x  Week_Number_x  Friday  Monday  Tuesday  Wednesday  Thursday  \
0    2019             14    0.47    0.39     0.38      -0.59     -0.39   
1    2019             15    0.59   -0.53     0.58      -0.75      0.11   
2    2019             16    0.40    0.40     0.83       0.00     -0.29   
3    2019             17    0.00   -1.35    -0.16       1.30     -0.72   
4    2019             19   -0.11   -0.97    -0.87      -1.20     -0.51   

                      Trend Expected Consenus_Trend lastday_Trend Year_Week  \
0        [Up, Up, Up, Down]     Down             Up          Down   2019_14   
1      [Up, Down, Up, Down]       Up           Flat          Flat   2019_15   
2        [Up, Up, Up, Flat]     Down             Up          Down   2019_16   
3    [Flat, Down, Down, Up]     Down           Down            Up   2019_17   
4  [Down, Down, Down, Down]     Down           Down          Down   2019_19   

   Year_y  Week_Number_y       pcr   pcr_vol  near_pcr  near_pcr_vol  
0    2019

In [195]:
result

,Year_x,Week_Number_x,Friday,Monday,Tuesday,Wednesday,Thursday,Trend,Expected,Consenus_Trend,lastday_Trend,Year_Week,Year_y,Week_Number_y,pcr,pcr_vol,near_pcr,near_pcr_vol
0,2019,14,0.47,0.39,0.38,-0.59,-0.39,"[Up, Up, Up, Down]",Down,Up,Down,2019_14,2019,14,1.268668,0.870815,1.068206,0.794347
1,2019,15,0.59,-0.53,0.58,-0.75,0.11,"[Up, Down, Up, Down]",Up,Flat,Flat,2019_15,2019,15,0.692563,0.823201,0.890929,1.661440
2,2019,16,0.40,0.40,0.83,0.00,-0.29,"[Up, Up, Up, Flat]",Down,Up,Down,2019_16,2019,16,1.632566,1.054964,1.061811,0.688205
3,2019,17,0.00,-1.35,-0.16,1.30,-0.72,"[Flat, Down, Down, Up]",Down,Down,Up,2019_17,2019,17,1.457176,0.893068,0.580621,0.275643
4,2019,19,-0.11,-0.97,-0.87,-1.20,-0.51,"[Down, Down, Down, Down]",Down,Down,Down,2019_19,2019,19,0.318322,0.671321,0.872058,2.741546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2024,21,0.28,0.00,0.12,0.31,1.64,"[Up, Flat, Up, Up]",Up,Up,Up,2024_21,2024,21,1.100094,0.835847,0.913113,0.528212
256,2024,22,-0.05,-0.11,-0.19,-0.80,-0.95,"[Flat, Down, Down, Down]",Down,Down,Down,2024_22,2024,22,0.704400,1.171862,0.979367,1.426766
257,2024,23,0.19,3.25,-5.93,3.36,0.89,"[Up, Up, Down, Up]",Up,Up,Up,2024_23,2024,23,0.535708,0.551546,0.791132,0.241626
258,2024,24,2.05,-0.13,0.02,0.25,0.33,"[Up, Down, Flat, Up]",Up,Up,Up,2024_24,2024,24,0.723595,0.697100,0.770853,0.528594


In [183]:
len(result[result['Expected'] == result['Consenus_Trend']])

115

In [ ]:
df_anomaly = result[result['Expected'] != result['Consenus_Trend']]
df_anomaly

In [194]:
df = df_anomaly[df_anomaly['Expected'] != df_anomaly['lastday_Trend']]
len(df)

93

In [193]:
len(df[(df['pcr'] > 0.8) & (df['pcr'] < 1.3)])

34

In [ ]:
result_df = result[[ 'Thursday', 'near_pcr', 'near_pcr_vol','Year_x']]
result_df.head(5)